In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model

np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
C = 10 # number of campaigns
U = 1000 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

In [3]:
#C = 10 # number of campaigns
#U = 10000 # number of customers.
#H = 3 # number of channels.
#D = 7 # number of planning days.
#I = 3 # number of quota categories.
#P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = [r_p[r] for r in np.random.choice(P, C)]

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
b = 7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
k = 3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
l_c = np.random.choice([2,3,4],C)

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
m_i = np.random.choice([4,3,5],I)
n_i = np.random.choice([1,3,2],I)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))

### Model

In [12]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [13]:
X_cuhd = {(c,u,h,d): mdl.binary_var(f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,C)
            for u in range(0,U) 
            for h in range(0,H)
            for d in range(0,D)}

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [14]:
maximize = mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * rp_c[c]
                  for c in range(0,C)
                  for u in range(0,U) 
                  for h in range(0,H) 
                  for d in range(0,D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [15]:
eligibilitiy = mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= e_cu[c,u]
    for c in range(0,C)
    for u in range(0,U) 
    for h in range(0,H) 
    for d in range(0,D))
)

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [16]:
one_channel = mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,H)) <= 1)
    for c in range(0,C)
    for u in range(0,U) 
    for d in range(0,D))
)

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [17]:
weekly_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,D) 
               for c in range(0,C) 
               for h in range(0,H)) <= b)
        for u in range(0,U)))

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [18]:
daily_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,C) 
               for h in range(0,H)) <= k)
        for d in range(0,D)
        for u in range(0,U)))

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [19]:
campaign_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,H) 
               for d in range(0,D)) <= l_c[c] )
        for c in range(0,C)
        for u in range(0,U)))

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [20]:
weekly_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C)
               for h in range(0,H) 
               for d in range(0,D)) <= m_i[i])
        for u in range(0,U)
        for i in range(0,I)))

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [21]:
daily_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C) 
               for h in range(0,H)) <= n_i[i])
        for u in range(0,U)
        for d in range(0,D)
        for i in range(0,I)))

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [22]:
channel_capacity = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,U) 
               for c in range(0,C)) <= t_hd[h,d])
        for h in range(0,H)
        for d in range(0,D)))

### Solution

In [23]:
%%time

#solution = mdl.solve(log_output=True, time_limit=600)
solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Found incumbent of value 0.000000 after 0.02 sec. (7.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 293504 rows and 116760 columns.
MIP Presolve modified 882 coefficients.
Reduced MIP has 28517 rows, 93240 columns, and 483210 nonzeros.
Reduced MIP has 93240 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.56 sec. (633.13 ticks)
Probing time = 0.49 sec. (23.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 58 rows and 42 columns.
Reduced MIP has 28459 rows, 93198 columns, and 481698 nonzeros.
Reduced MIP has 93198 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.11 sec. (480.29 ticks)
Probing time = 0.12 sec. (23.26 ticks)
Clique table members: 20272.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode

In [24]:
solution.objective_value

262149.0

In [25]:
solution.as_df().sum()["value"]

6658.0

In [26]:
X_cuhd2 = np.zeros((C,U,H,D), dtype='int')
for ky,_ in solution.as_name_dict().items():
    exec(f'X_cuhd2{[int(i.split(":")[1]) for i in ky.split("_")[1:]]} = 1', {}, {'X_cuhd2':X_cuhd2})

In [27]:
eligibility = lambda X, c, u, h, d: X[c,u,h,d]<=e_cu[c,u]
one_channel = lambda X, c, u, d: X[c,u,:,d].sum() <= 1
weekly_limitation = lambda X, u: X[:,u,:,:].sum() <= b
daily_limitation = lambda X, u, d: X[:,u,:,d].sum() <= k
campaign_limitation = lambda X, c, u: X[c,u,:,:].sum() <= l_c[c]
weekly_quota = lambda X, u: all((q_ic * X[:,u,:,:].sum(axis=(1,2))).sum(axis=1)<=m_i)
daily_quota = lambda X, u, d: all((q_ic * X[:,u,:,d].sum(axis=(1))).sum(axis=1)<=n_i)
channel_capacity = lambda X, h, d: X[:,:,h,d].sum() <= t_hd[h,d]

class Constraint:
    def __init__(self, name, fnc, fixed_indicies):
        self.fnc = fnc
        self.name = name
        self.fixed_indicies = fixed_indicies
        self.exec_count = 0

    def execute(self, X, indicies):
        self.exec_count = self.exec_count + 1
        params = tuple([X]) + tuple([indicies[fi] for fi in self.fixed_indicies])
        return self.fnc(*params)
    
    def get_exec_count(self):
        return self.exec_count


class Model:
    def __init__(self, constraints):
        self.constraints = constraints
        
    def execute(self, X, indicies):
        for c in self.constraints:
            if not c.execute(X, indicies):
                return False
        return True
    
    def stats(self):
        return [(c.name, c.get_exec_count()) for c in self.constraints]
            
c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = Model([
    Constraint('eligibility',eligibility, (c_i, u_i, h_i, d_i,)),
    Constraint('channel_capacity',channel_capacity, (h_i, d_i,)),
    Constraint('daily_limitation',daily_limitation, (u_i, d_i,)),
    Constraint('weekly_limitation',weekly_limitation, (u_i,)),
    Constraint('campaign_limitation',campaign_limitation, (c_i, u_i,)),
    Constraint('daily_quota',daily_quota, (u_i, d_i,)),
    Constraint('one_channel',one_channel, (c_i, u_i, d_i,)),
    Constraint('weekly_quota',weekly_quota, (u_i,))
])
def validate():
    for c in range(C):
        for d in range(D):
            for h in range(H):
                for u in range(U):
                    if not mdl.execute(X_cuhd2, (c, u, h, d)):
                        print('!!!! ->',(c, u, h, d))
validate()

np.matmul(rp_c, X_cuhd2.sum(axis=(1,2,3)))

262149